In [1]:
import numpy as np
from decimal import Decimal

import nbimporter
from DirectionCoefficient import get_nu_H as DC_get_nu_H
from DirectionCoefficient import get_nu_C as DC_get_nu_C
from fValue import f1f2Value

Importing Jupyter notebook from DirectionCoefficient.ipynb
Importing Jupyter notebook from fValue.ipynb


# Envelope Performance / 外皮性能

## Envelope Evaluation Method by Not Using Area of Envelope Units of Actual House / 当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法

## 1. Introduction

This calculation depends on the specification of "Chapter III Heating / Cooling Load and Envelope Performance - II Envelope Performance" in BRI.

## 2. Type

### 2.1 Region / 地域区分 

Region consists of 'region1', 'region2', 'region3', ... , 'region8' as string.

### 2.2 Direction / 方位

Direction consists of 'S', 'SW', 'W', 'NW', 'N', 'NE', 'E', 'SE', 'top' and 'bottom' as string.

### 2.3 Type of Reference House for Simple Evaluation / 簡易法における標準住戸の種類 

The type of reference house for simple evaluation consists below;  
* floor_ins : Floor insulated house
* base_ins  : Base insulated house
* floor_and_base_ins : The house which hase the parts of both floor insulated part and base insulated part

## 3. Functions

### 3.1. Heat Loss Coefficient without Ventilation Loss / 熱損失係数(換気による熱損失を含まない)

$ Q' = U_A \times r_{env} $

$Q'$ is the heat loss coefficient without ventilation loss / 熱損失係数(換気による熱損失を含まない) (W/m<sup>2</sup>K)  
$U_A$ is the average envelope thermal transmittance rate / 外皮平均熱貫流率(W/m<sup>2</sup>K)  
$r_{env}$ is the ratio of the total envelope parts area to the total floor area / 床面積の合計に対する外皮の部位の面積の合計の比

In [2]:
def get_Q_dash(U_A, r_env):
    return U_A * r_env

#### Comment

The average envelope thermal transmittance rate is the value of the heat loss per the unit temperature difference $q$ divided by the total floor area $ A_f $.  
$ U_A = q \div A_f $  
The heat loss coefficient withou ventilation loss is tha value of the heat loss per the unit temperature difference $q$ divided by the total envelope parts area $ A_{env} $.  
$ Q' = q \div A_{env} $  
The ratio of the total envelope parts area to the total floor area $r_{env}$ is the coefficient for converting $U_A$ value to $Q'$ value.

#### Example

In [3]:
get_Q_dash( 0.87 , 307.51/120.08 )

2.227962191872085

### 3.2 Solar Gain Coefficient / 日射取得係数

$ \mu_H = \frac { \eta_{A,H} }{100} \times r_{env} $

$ \mu_C = \frac { \eta_{A,C} }{100} \times r_{env} $

$ \mu_H $ is the solar gain coefficient in the heating season / 暖房期の日射取得係数;  
$ \mu_C $ is the solar gain coefficient in the cooling season / 冷房期の日射取得係数;  
$ \eta_{A,H} $ is the average solar heat gain ratio in the heating season / 暖房期の平均日射熱取得率 (%);  
$ \eta_{A,C} $ is the average solar heat gain ratio in the cooling season / 冷房期の平均日射熱取得率 (%);  
$r_{env}$ is the ratio of the total envelope parts area to the total floor area / 床面積の合計に対する外皮の部位の面積の合計の比.

In [4]:
def get_mu_H(eta_A_H, r_env):
    return eta_A_H * r_env / 100

In [5]:
def get_mu_C(eta_A_C, r_env):
    return eta_A_C * r_env / 100

#### Example

In [6]:
get_mu_H(4.3, 307.51/120.08 )

0.11011767155229846

In [7]:
get_mu_C(2.8, 307.51/120.08 )

0.07170453031312457

### 3.3 Ratio of Total Envelope Parts Area to Total Floor Area  / 床面積の合計に対する外皮の部位の面積の合計の比

$ \displaystyle r_{env} = \frac{ A_{env} }{ A_A } $ 

$r_{env}$ is the ratio of the total envelope parts area to the total floor area / 床面積の合計に対する外皮の部位の面積の合計の比;  
$A_{env}$ is the total envelope parts area / 外皮の部位の面積の合計 (m<sup>2</sup>);  
$A_A$ is the total floor area / 床面積の合計 (m<sup>2</sup>).

In [8]:
def get_r_env(A_env, A_A):
    return A_env / A_A

#### Example

In [9]:
get_r_env( 307.51, 120.08 )

2.560876082611592

### 3.4 Average Envelope Thermal Transmittance Rate / 外皮平均熱貫流率

Where there are any fractions after two decimal places of $U_A$ value, they shall be rounded up. 

$ \displaystyle U_A = \frac{ q }{ A_{env} } $

$U_A$ is the average envelope thermal transmittance rate / 外皮平均熱貫流率 (W/m<sup>2</sup>K);  
$q$ is the envelope heat loss per unit temperature difference / 単位温度差当たりの外皮熱損失量(W/K);   
$A_{env}$ is the total envelope parts area / 外皮の部位の面積の合計 (m<sup>2</sup>).

$ \begin{align}
    q &= A_{roof} \times H_{roof} \times U_{roof} \\
      &+ ( A_{wall,0} + A_{wall,90} + A_{wall,180} + A_{wall,270} ) \times H_{wall} \times U_{wall} \\
      &+ ( A_{door,0} + A_{door,90} + A_{door,180} + A_{door,270} ) \times H_{door} \times U_{door} \\
      &+ ( A_{wnd,0}  + A_{wnd,90}  + A_{wnd,180}  + A_{wnd,270}  ) \times H_{wnd}  \times U_{wnd}  \\
      &+ A_{floor,o} \times H_{floor} \times U_{floor,o} \\
      &+ A_{floor,b} \times H_{floor} \times U_{floor,b} \\
      &+ ( ( A_{base,o,OS,0} + A_{base,o,OS,90} + A_{base,o,OS,180} + A_{base,o,OS,270} ) \times H_{base,OS} + A_{base,o,IS} \times H_{base,IS} ) \times U_{base,o} \\
      &+ ( ( A_{base,e,OS,0} + A_{base,e,OS,90} + A_{base,e,OS,180} + A_{base,e,OS,270} ) \times H_{base,OS} + A_{base,e,IS} \times H_{base,IS} ) \times U_{base,e} \\
      &+ ( ( A_{base,b,OS,0} + A_{base,b,OS,90} + A_{base,b,OS,180} + A_{base,b,OS,270} ) \times H_{base,OS} + A_{base,b,IS} \times H_{base,IS} ) \times U_{base,b} \\
      &+ ( ( L_{prm,o,OS,0} + L_{prm,o,OS,90} + L_{prm,o,OS,180} + L_{prm,o,OS,270} ) \times H_{prm,OS} + L_{prm,o,IS} \times H_{prm,IS} ) \times \psi_{prm,o} \\
      &+ ( ( L_{prm,e,OS,0} + L_{prm,e,OS,90} + L_{prm,e,OS,180} + L_{prm,e,OS,270} ) \times H_{prm,OS} + L_{prm,e,IS} \times H_{prm,IS} ) \times \psi_{prm,e} \\
      &+ ( ( L_{prm,b,OS,0} + L_{prm,b,OS,90} + L_{prm,b,OS,180} + L_{prm,b,OS,270} ) \times H_{prm,OS} + L_{prm,b,IS} \times H_{prm,IS} ) \times \psi_{prm,b} \\
\end{align}$  

$ A $ is the area / 面積 (m<sup>2</sup>);  
$ L $ is the length / 長さ (m);  
$ H $ is the temperature difference coefficient / 温度差係数;  
$ U $ is the thermal transmittance rate / 熱貫流率 (W/m<sup>2</sup>K);  
$ \psi $ is the linear thermal transmittance rate / 線熱貫流率 (W/mK).  

subscript  
roof : the roof / 屋根;  
wall : the (external) wall / 壁;  
door : the door / ドア;  
wnd : the window / 窓;  
floor : the (external) floor / 床;  
base : the base / 基礎;  
prm : the perimeter of the earth floor / 土間床外周部;  
e : the entrance / 玄関部分;  
b : the bath / 浴室;  
o : the part except the entrance and the bath / 玄関部分及び浴室を除く部分;  
0,90,180,270 : the direction of 0, 90, 180 and 270 degree of clockwise from the south / 南から時計回りに0,90,180,270度の方向;  
OS,IS : the outside and inside / 室外側及び室内側.

In [10]:
class Orientation():
    def __init__(self,_D0,_D90,_D180,_D270):
        self.D0   = _D0;
        self.D90  = _D90;
        self.D180 = _D180;
        self.D270 = _D270;
    def sum(self):
        return self.D0 + self.D90 + self.D180 + self.D270
    def list(self):
        return [self.D0, self.D90, self.D180, self.D270]
    def __mul__(x,y):
        return Orientation(x.D0 * y.D0, x.D90 * y.D90, x.D180 * y.D180, x.D270 * y.D270)

D0:   direction of   0 degree of clockwise from the south    
D90:  direction of  90 degree of clockwise from the south  
D180: direction of 180 degree of clockwise from the south  
D270: direction of 270 degree of clockwise from the south  

In [11]:
def get_simple_U_A (A, L, H, U, psi):

    # A : Area
    #     roof, floor_o, base_o_IS, base_e_IS, total_env : float
    #     wall, door, window, base_o_OS, base_e_OS : class 'Orientation' which has D0,D90,D180 and D270 as float
    # L : Length
    #     prm_o_IS, prm_e_IS : float
    #     prm_o_OS, prm_e_OS : class 'Orientation' which has D0,D90,D180 and D270 as float
    # H : Temperature difference coefficient
    #     roof, wall, door, window, floor, base_OS, base_IS, prm_OS, prm_IS: float
    # U : U-Value
    #     roof, wall, door, window, floorOther, baseOther, baseEntrance: float
    # psi : psi value
    #     perimeterOther, perimeterEntrance : float

    q = A['roof']         * H['roof']   * U['roof']   \
      + A['wall'].sum()   * H['wall']   * U['wall']   \
      + A['door'].sum()   * H['door']   * U['door']   \
      + A['window'].sum() * H['window'] * U['window'] \
      + A['floor_o']      * H['floor']  * U['floorOther'] \
      + ( A['base_o_OS'].sum() * H['base_OS'] + A['base_o_IS'] * H['base_IS'] ) * U['baseOther']  \
      + ( A['base_e_OS'].sum() * H['base_OS'] + A['base_e_IS'] * H['base_IS'] ) * U['baseEntrance']  \
      + ( L['prm_o_OS'].sum()  * H['prm_OS']  + L['prm_o_IS']  * H['prm_IS']  ) * psi['perimeterOther'] \
      + ( L['prm_e_OS'].sum()  * H['prm_OS']  + L['prm_e_IS']  * H['prm_IS']  ) * psi['perimeterEntrance']        
    
    return float(Decimal(q / A['total_env']).quantize(Decimal('0.00'), rounding='ROUND_UP'))

#### Example

In [12]:
get_simple_U_A ( A = {
                     'roof'      : 50.85,
                     'wall'      : Orientation(30.47, 22.37, 47.92, 22.28),
                     'door'      : Orientation(0.0, 1.89, 1.62, 0.0),
                     'window'    : Orientation(22.69, 2.38, 3.63, 4.37),
                     'floor_o'   : 45.05,
                     'base_o_OS' : Orientation(0.00, 0.91, 0.91, 0.00),
                     'base_o_IS' : 1.82, 
                     'base_e_OS' : Orientation(0.00, 0.33, 0.25, 0.00),
                     'base_e_IS' : 0.57,
                     'total_env' : 266.10
                 },
                 L = {
                     'prm_o_OS' : Orientation(  0.00, 1.82, 1.82, 0.00 ),
                     'prm_o_IS' : 3.64,
                     'prm_e_OS' : Orientation(  0.00, 1.82, 1.37, 0.00 ),
                     'prm_e_IS' : 3.19
                 },
                 H = {
                     'roof'    : 1.0,
                     'wall'    : 1.0,
                     'door'    : 1.0,
                     'window'  : 1.0,
                     'floor'   : 0.7,
                     'base_OS' : 1.0,
                     'base_IS' : 0.7,
                     'prm_OS'  : 1.0,
                     'prm_IS'  : 0.7
                 },
                 U = {
                     'roof'         : 1.0,
                     'wall'         : 1.0,
                     'door'         : 1.0,
                     'window'       : 1.0,
                     'floorOther'   : 1.0,
                     'baseOther'    : 1.0,
                     'baseEntrance' : 1.0,
                 },
                 psi = {
                     'perimeterOther'    : 1.0,
                     'perimeterEntrance' : 1.0
                 }
               )                    

0.97

### 3.5 Average Solar Heat Gain Ratio in Heating and Cooling Season / 暖房期の平均日射熱取得率及び冷房期の平均日射熱取得率

Where there are any fractions after one decimal places of $\eta_{A,H}$, they shall be rounded down. 

$ \displaystyle \eta_{A,H} = \frac{m_H}{A_{env}} \times 100 $

$ \eta_{A,H} $ is the average solar heat gain ratio in the heating season / 暖房期の平均日射熱取得率 (%);  
$ m_H $ is the solar heat gain in the heating season per unit solar radiation intensity / 単位日射強度当たりの暖房期の日射熱取得量(W/(W/m<sup>2</sup>));  
$ A_{env} $ is the total envelope parts area / 外皮の部位の面積の合計 (m<sup>2</sup>).

Where there are any fractions after one decimal places of $\eta_{A,C}$, they shall be rounded up. 

$ \displaystyle \eta_{A,C} = \frac{m_C}{A_{env}} \times 100 $

$ \eta_{A,C} $ is the average solar heat gain ratio in the cooling season / 冷房期の平均日射熱取得率 (%);  
$ m_C $ is the solar heat gain in the cooling season per unit solar radiation intensity / 単位日射強度当たりの冷房期の日射熱取得量(W/(W/m<sup>2</sup>)).

$ \begin{align}
    m_H &= A_{roof} \times \nu_{H,top} \times \eta_{H,roof} \\
        &+ ( A_{wall,0} \times \nu_{H,0} + A_{wall,90} \times \nu_{H,90} + A_{wall,180} \times \nu_{H,180} + A_{wall,270} \times \nu_{H,270} ) \times \eta_{H,wall} \\
        &+ ( A_{door,0} \times \nu_{H,0} + A_{door,90} \times \nu_{H,90} + A_{door,180} \times \nu_{H,180} + A_{door,270} \times \nu_{H,270} ) \times \eta_{H,door} \\
        &+ A_{wnd,0} \times \nu_{H,0} \times \eta_{H,wnd,0} + A_{wnd,90} \times \nu_{H,90} \times \eta_{H,wnd,90} \\
        &+ A_{wnd,180} \times \nu_{H,180} \times \eta_{H,wnd,180} + A_{wnd,270} \times \nu_{H,270} \times \eta_{H,wnd,270} \\
        &+ ( A_{base,o,0} \times \nu_{H,0} + A_{base,o,90} \times \nu_{H,90} + A_{base,o,180} \times \nu_{H,180} + A_{base,o,270} \times \nu_{H,270} ) \times \eta_{H,base,o} \\
        &+ ( A_{base,e,0} \times \nu_{H,0} + A_{base,e,90} \times \nu_{H,90} + A_{base,e,180} \times \nu_{H,180} + A_{base,e,270} \times \nu_{H,270} ) \times \eta_{H,base,e} \\
\end{align}$  

$ \begin{align}
    m_C &= A_{roof} \times \nu_{C,top} \times \eta_{C,roof} \\
        &+ ( A_{wall,0} \times \nu_{C,0} + A_{wall,90} \times \nu_{C,90} + A_{wall,180} \times \nu_{C,180} + A_{wall,270} \times \nu_{C,270} ) \times \eta_{C,wall} \\
        &+ ( A_{door,0} \times \nu_{C,0} + A_{door,90} \times \nu_{C,90} + A_{door,180} \times \nu_{C,180} + A_{door,270} \times \nu_{C,270} ) \times \eta_{C,door} \\
        &+ A_{wnd,0} \times \nu_{C,0} \times \eta_{C,wnd,0} + A_{wnd,90} \times \nu_{C,90} \times \eta_{C,wnd,90} \\
        &+ A_{wnd,180} \times \nu_{C,180} \times \eta_{C,wnd,180} + A_{wnd,270} \times \nu_{C,270} \times \eta_{C,wnd,270} \\
        &+ ( A_{base,o,0} \times \nu_{C,0} + A_{base,o,90} \times \nu_{C,90} + A_{base,o,180} \times \nu_{C,180} + A_{base,o,270} \times \nu_{C,270} ) \times \eta_{C,base,o} \\
        &+ ( A_{base,e,0} \times \nu_{C,0} + A_{base,e,90} \times \nu_{C,90} + A_{base,e,180} \times \nu_{C,180} + A_{base,e,270} \times \nu_{C,270} ) \times \eta_{C,base,e}
\end{align}$  

$ A $ is the area / 面積 (m<sup>2</sup>);  
$ \nu $ is the direction coefficient / 方位係数;  
$ \eta $ is the solar heat gain ratio / 日射熱取得率  

subscript  
roof : the roof / 屋根;  
wall : the (external) wall / 壁;  
door : the door / ドア;  
wnd : the window / 窓;  
base : the base / 基礎;  
e : the entrance / 玄関部分;  
b : the bath / 浴室;  
o : the part except the entrance and the bath / 玄関部分及び浴室を除く部分;  
0,90,180,270 : the direction of 0, 90, 180 and 270 degree of clockwise from the south / 南から時計回りに0,90,180,270度の方向;  
top : the direction of the top / 上向き方向;  
OS,IS : the outside and inside / 室外側及び室内側;  
H : heating season / 暖房期;  
C : cooling season / 冷房期.  

In [13]:
def get_simple_eta_A_H(A, nu_H_top, nu_H, eta_H_roof, eta_H_wall, eta_H_door, eta_H_wnd, eta_H_base_o, eta_H_base_e):
    # A : Area
    #     roof, total_env : float
    #     wall, door, window, base_o_OS, base_e_OS : class 'Orientation' which has D0,D90,D180 and D270 as float
    # nu_H_top; float
    # nu_H: class 'Orientation' which has D0,D90,D180 and D270 as float
    # eta_H_roof, eta_H_wall, eta_H_door, eta_H_base, eta_H_base_d: float
    # eta_H_wnd: class 'Orientation' which has D0,D90,D180 and D270 as float
    # A_env: float
    m_H = A['roof'] * nu_H_top * eta_H_roof \
        + ( A['wall']      * nu_H ).sum() * eta_H_wall \
        + ( A['door']      * nu_H ).sum() * eta_H_door \
        + ( A['window']    * nu_H * eta_H_wnd ).sum() \
        + ( A['base_o_OS'] * nu_H ).sum() * eta_H_base_o \
        + ( A['base_e_OS'] * nu_H ).sum() * eta_H_base_e

    return float(Decimal( m_H / A['total_env'] * 100 ).quantize(Decimal('0.0'), rounding = 'ROUND_DOWN'))

In [14]:
def get_simple_eta_A_C(A, nu_C_top, nu_C, eta_C_roof, eta_C_wall, eta_C_door, eta_C_wnd, eta_C_base_o, eta_C_base_e):
    # A : Area
    #     roof, total_env : float
    #     wall, door, window, base_o_OS, base_e_OS : class 'Orientation' which has D0,D90,D180 and D270 as float
    # nu_C_top; float
    # nu_C: class 'Orientation' which has D0,D90,D180 and D270 as float
    # eta_C_roof, eta_H_wall, eta_H_door, eta_H_base, eta_H_base_d: float
    # eta_C_wnd: class 'Orientation' which has D0,D90,D180 and D270 as float
    # A_env: float
    m_C = A['roof'] * nu_C_top * eta_C_roof \
        + ( A['wall']      * nu_C ).sum() * eta_C_wall \
        + ( A['door']      * nu_C ).sum() * eta_C_door \
        + ( A['window']    * nu_C * eta_C_wnd ).sum() \
        + ( A['base_o_OS'] * nu_C ).sum() * eta_C_base_o \
        + ( A['base_e_OS'] * nu_C ).sum() * eta_C_base_e

    return float(Decimal( m_C / A['total_env'] * 100 ).quantize(Decimal('0.0'), rounding = 'ROUND_UP'))

#### Example

In [15]:
get_simple_eta_A_H({
                     'roof'      : 50.85,
                     'wall'      : Orientation(30.47, 22.37, 47.92, 22.28),
                     'door'      : Orientation( 0.00,  1.89,  1.62,  0.00),
                     'window'    : Orientation(22.69,  2.38,  3.63,  4.37),
                     'base_o_OS' : Orientation( 0.00,  0.91,  0.91,  0.00),
                     'base_e_OS' : Orientation( 0.00,  0.33,  0.25,  0.00),
                     'total_env' : 266.10
                 }, 1.0, Orientation(0.763, 0.317, 0.325, 0.833), 0.034, 0.034, 0.034, Orientation(0.5, 0.5, 0.5, 0.5), 0.034, 0.034)

5.7

In [16]:
get_simple_eta_A_C({
                     'roof'      : 50.85,
                     'wall'      : Orientation(30.47, 22.37, 47.92, 22.28),
                     'door'      : Orientation( 0.00,  1.89,  1.62,  0.00),
                     'window'    : Orientation(22.69,  2.38,  3.63,  4.37),
                     'base_o_OS' : Orientation( 0.00,  0.91,  0.91,  0.00),
                     'base_e_OS' : Orientation( 0.00,  0.33,  0.25,  0.00),
                     'total_env' : 266.10
                 }, 1.0, Orientation(0.491, 0.427, 0.431, 0.498), 0.034, 0.034, 0.034, Orientation(0.5, 0.5, 0.5, 0.5), 0.034, 0.034)

4.4

### 3.6 Direction Coefficient / 方位係数

Main direction is the southwest.

In [17]:
def get_simple_Orientation_value_from_Direction(direction):
    # direction: [(S,SW,W,NW,N,NE,E,SE,top,bottom)の辞書型]
    return Orientation( direction['SW'], direction['NW'], direction['NE'], direction['SE'] )

### 3.7 Area of Envelope parts and Perimeter Length of Earth Floor for Referenced House / 標準住戸における外皮の部位の面積及び土間床等の外周部の長さ等

In [18]:
def getSimpleAreaAndLength(houseType):
    area = {
        'floor_ins': {
            'roof'        : 50.85,
            'wall'        : Orientation( 30.47, 22.37, 47.92, 22.28 ),
            'door'        : Orientation(  0.00,  1.89,  1.62,  0.00 ),
            'window'      : Orientation( 22.69,  2.38,  3.63,  4.37 ),
            'floor_o'     : 45.05,
            'base_o_OS'   : Orientation(  0.00,  0.91,  0.91,  0.00 ),
            'base_o_IS'   : 1.82,
            'base_e_OS'   : Orientation(  0.00,  0.33,  0.25,  0.00 ),
            'base_e_IS'   : 0.57,
            'total_env'   : 266.10,
            'total_floor' : 90.0
        },
        'base_ins': {
            'roof'        : 50.85,
            'wall'        : Orientation( 30.47, 22.37, 47.92, 22.28 ),
            'door'        : Orientation(  0.00,  1.89,  1.62,  0.00 ),
            'window'      : Orientation( 22.69,  2.38,  3.63,  4.37 ),
            'floor_o'     : 0.0,
            'base_o_OS'   : Orientation(  5.30,  1.48,  4.62,  2.40 ),
            'base_o_IS'   : 0.0,
            'base_e_OS'   : Orientation(  0.00,  0.33,  0.25,  0.00 ),
            'base_e_IS'   : 0.00,
            'total_env'   : 275.69,
            'total_floor' : 90.0
        }
    }[houseType]
    length = {
        'floor_ins': {
            'prm_o_OS' : Orientation(  0.00,  1.82,  1.82,  0.00 ),
            'prm_o_IS' : 3.64,
            'prm_e_OS' : Orientation(  0.00,  1.82,  1.37,  0.00 ),
            'prm_e_IS' : 3.19
        },
        'base_ins': {
            'prm_o_OS' : Orientation( 10.61,  2.97,  9.24,  4.79 ),
            'prm_o_IS' : 0.00,
            'prm_e_OS' : Orientation(  0.00,  1.82,  1.37,  0.00 ),
            'prm_e_IS' : 0.00
        }
    }[houseType]
    return ( area, length )

### 3.8 Judgement of Applyment of Floor Insulated House or Base Insulated House for Evaluation / 床断熱住宅か基礎断熱住宅の判断

When the house is 'floor_and_base_ins' house, decide which 'floor_ins' house or 'base_ins' house is applyed for this simple evaluation base on the UA value. The house which has the larger UA shall be applied. 

In [19]:
def judge_simple_house_type(floor_ins_U_A, base_ins_U_A):
    return (floor_ins_U_A, 'floor_ins') if floor_ins_U_A > base_ins_U_A else (base_ins_U_A, 'base_ins')

#### Example

The case that UA of the floor insulated house is larger.

In [20]:
judge_simple_house_type(2.2, 1.1)

(2.2, 'floor_ins')

The case that UA of the base insulated house is larger.

In [21]:
judge_simple_house_type(1.1, 2.2)

(2.2, 'base_ins')

### 3.9 Temperature Difference Coefficient / 外皮の部位及び土間床等の周辺部の温度差係数

| part | H |
| --- | --- |
| roof | 1.0 |
| wall | 1.0 |
| door | 1.0 |
| window | 1.0 |
| floor | 1.0 |
| basement to Outdoor Side | 1.0 |
| basement to Internal Side | 1.0 |
| perimeter to Outdoor Side | 1.0 |
| perimeter to Internal Side | 1.0 |


In [22]:
def getSimpleHValue():
    return {
        'roof'    : 1.0,
        'wall'    : 1.0,
        'door'    : 1.0,
        'window'  : 1.0,
        'floor'   : 0.7,
        'base_OS' : 1.0,
        'base_IS' : 0.7,
        'prm_OS'  : 1.0,
        'prm_IS'  : 0.7
    }

### 3.10 Linear Thermal Transmittance Rate of Perimeter of Earth Floor / 土間床外周部の線熱貫流率

The defalut value is 1.8 (W/mK)

In [23]:
def setPsiPerimeter(psi):
    if psi['useDefault'] == 'yes':
        psi['perimeterEntrance'] = 1.8
        psi['perimeterOther']    = 1.8
    return psi

### 3.11 Solar Heat Gain Ratio of Envelope Parts except for Windows / 外皮の部位(窓を除く)の日射熱取得率

$ \eta_{H,roof} = 0.034 \times U_{roof} \times \gamma_H $  
$ \eta_{C,roof} = 0.034 \times U_{roof} \times \gamma_C $  
$ \eta_{H,wall} = 0.034 \times U_{wall} \times \gamma_H $  
$ \eta_{C,wall} = 0.034 \times U_{wall} \times \gamma_C $  
$ \eta_{H,door} = 0.034 \times U_{door} \times \gamma_H $  
$ \eta_{C,door} = 0.034 \times U_{door} \times \gamma_C $  
$ \eta_{H,base,O} = 0.034 \times U_{base,O} \times \gamma_H $  
$ \eta_{C,base,O} = 0.034 \times U_{base,O} \times \gamma_C $  
$ \eta_{H,base,e} = 0.034 \times U_{base,e} \times \gamma_H $  
$ \eta_{C,base,e} = 0.034 \times U_{base,e} \times \gamma_C $  


$ \eta $ is the solar heat gain ratio / 日射熱取得率  
$ U $ is the thermal transmittance rate / 熱貫流率 (W/m<sup>2</sup>K);  

subscript  
roof : the roof / 屋根;  
wall : the (external) wall / 壁;  
door : the door / ドア;  
base : the base / 基礎;  
H : heating season / 暖房期;  
C : cooling season / 冷房期.  
b : the bath / 浴室;  
e : the entrance / 玄関部分;  
o : the part except the entrance and the bath / 玄関部分及び浴室を除く部分;  

In [24]:
def get_simple_eta_not_window(U_part):
    gamma_H = 1.0 # 暖房期の日除けの効果係数
    gamma_C = 1.0 # 冷房期の日除けの効果係数
    return ( 0.034 * U_part * gamma_H, 0.034 * U_part * gamma_C)

#### Example

$U$ = 1.0

In [25]:
get_simple_eta_not_window(1.0)

(0.034, 0.034)

### 3.12 Solar Heat Gain Ratio of Windows / 窓の日射熱取得率

#### Heating Season

$ \eta_{H,window,0} = \eta_{d,H} \times f_{H,0} $  
$ \eta_{H,window,90} = \eta_{d,H} \times f_{H,90} $  
$ \eta_{H,window,180} = \eta_{d,H} \times f_{H,180} $  
$ \eta_{H,window,270} = \eta_{d,H} \times f_{H,270} $  

$ \eta_{H,window} $ is the solar heat gain ratio of the windows in heating season / 暖房期の窓の日射熱取得率  
$ \eta_d $ is the solar heat gain ratio of the windows for prependicular solar direction / 垂直面日射熱取得率  
$ f_H $ is the correction factor for the solar heat gain of the windows in heating season / 暖房期の窓の取得日射熱補正係数

subscript  
0,90,180,270 : the direction of 0, 90, 180 and 270 degree of clockwise from the south / 南から時計回りに0,90,180,270度の方向;  

In case of using the default value for $f_H$, $f_H$ shall be calculated in the condition that $l_1 = 0$, $l_2 = 1 \div 0.3$, the glass type is 'Type7' for the each direction of the southwest, the northwest, the northeast and the southeast in each region. 

In [26]:
def get_f_H_default(region):
    my_round = lambda x: (x * 1000 * 2 + 1) // 2 / 1000
    v = f1f2Value()
    return Orientation(
        my_round( v.getValue(glassType = 'Type7', season = 'Heating', region = region, direction = 'SW', lValue = 1/0.3) ),
        my_round( v.getValue(glassType = 'Type7', season = 'Heating', region = region, direction = 'NW', lValue = 1/0.3) ),
        my_round( v.getValue(glassType = 'Type7', season = 'Heating', region = region, direction = 'NE', lValue = 1/0.3) ),
        my_round( v.getValue(glassType = 'Type7', season = 'Heating', region = region, direction = 'SE', lValue = 1/0.3) )
    )

f_H value in each regions.

In [27]:
for r in ['region1','region2','region3','region4','region5','region6','region7']:
    print( str(r) + ': ' + str(vars(get_f_H_default(r)) ))

region1: {'D0': 0.658, 'D90': 0.605, 'D180': 0.602, 'D270': 0.665}
region2: {'D0': 0.648, 'D90': 0.604, 'D180': 0.605, 'D270': 0.656}
region3: {'D0': 0.65, 'D90': 0.602, 'D180': 0.602, 'D270': 0.649}
region4: {'D0': 0.642, 'D90': 0.595, 'D180': 0.598, 'D270': 0.642}
region5: {'D0': 0.667, 'D90': 0.595, 'D180': 0.592, 'D270': 0.671}
region6: {'D0': 0.654, 'D90': 0.595, 'D180': 0.589, 'D270': 0.675}
region7: {'D0': 0.67, 'D90': 0.589, 'D180': 0.587, 'D270': 0.666}


In [28]:
def get_simple_eta_H_window(eta_d_H, f_H, is_f_value_default, region):
    if is_f_value_default == True :
        default_f_H = get_f_H_default(region)
        return Orientation(eta_d_H * default_f_H.D0, eta_d_H * default_f_H.D90, eta_d_H * default_f_H.D180, eta_d_H * default_f_H.D270)
    else:
        return Orientation(eta_d_H * f_H, eta_d_H * f_H, eta_d_H * f_H, eta_d_H * f_H)

#### Example

Use default value for f_H

In [29]:
vars(get_simple_eta_H_window(0.5, 0.7, True, 'region6'))

{'D0': 0.327, 'D180': 0.2945, 'D270': 0.3375, 'D90': 0.2975}

Not use default value for f_H

In [30]:
vars(get_simple_eta_H_window(0.5, 0.7, False, 'region6'))

{'D0': 0.35, 'D180': 0.35, 'D270': 0.35, 'D90': 0.35}

#### Cooling Season

$ \eta_{C,window,0} = \eta_{d,C} \times f_{C,0} $  
$ \eta_{C,window,90} = \eta_{d,C} \times f_{C,90} $  
$ \eta_{C,window,180} = \eta_{d,C} \times f_{C,180} $  
$ \eta_{C,window,270} = \eta_{d,C} \times f_{C,270} $  

$ \eta_{C,window} $ is the solar heat gain ratio of the windows in cooling season / 冷房期の窓の日射熱取得率  
$ \eta_d $ is the solar heat gain ratio of the windows for prependicular solar direction / 垂直面日射熱取得率  
$ f_C $ is the correction factor for the solar heat gain of the windows in cooling season / 冷房期の窓の取得日射熱補正係数

subscript  
0,90,180,270 : the direction of 0, 90, 180 and 270 degree of clockwise from the south / 南から時計回りに0,90,180,270度の方向;  

In case of using the default value for $f_C$, $f_C$ shall be calculated in the condition that $l = 20$, the glass type is 'Type1' for the each direction of the southwest, the northwest, the northeast and the southeast in each region. 

In [31]:
def get_f_C_default(region):
    my_round = lambda x: (x * 1000 * 2 + 1) // 2 / 1000
    v = f1f2Value()
    return Orientation(
        my_round( v.getValue(glassType = 'Type1', season = 'Cooling', region = region, direction = 'SW', lValue = 20.0) ),
        my_round( v.getValue(glassType = 'Type1', season = 'Cooling', region = region, direction = 'NW', lValue = 20.0) ),
        my_round( v.getValue(glassType = 'Type1', season = 'Cooling', region = region, direction = 'NE', lValue = 20.0) ),
        my_round( v.getValue(glassType = 'Type1', season = 'Cooling', region = region, direction = 'SE', lValue = 20.0) ),
    )

f_C value in each regions.

In [32]:
for r in ['region1','region2','region3','region4','region5','region6','region7','region8']:
    print( str(r) + ': ' + str(vars(get_f_C_default(r))) )

region1: {'D0': 0.86, 'D90': 0.866, 'D180': 0.865, 'D270': 0.864}
region2: {'D0': 0.861, 'D90': 0.864, 'D180': 0.864, 'D270': 0.858}
region3: {'D0': 0.859, 'D90': 0.865, 'D180': 0.862, 'D270': 0.853}
region4: {'D0': 0.85, 'D90': 0.861, 'D180': 0.861, 'D270': 0.853}
region5: {'D0': 0.858, 'D90': 0.862, 'D180': 0.863, 'D270': 0.854}
region6: {'D0': 0.852, 'D90': 0.864, 'D180': 0.862, 'D270': 0.852}
region7: {'D0': 0.847, 'D90': 0.862, 'D180': 0.861, 'D270': 0.849}
region8: {'D0': 0.858, 'D90': 0.859, 'D180': 0.857, 'D270': 0.86}


In [33]:
def get_simple_eta_C_window(eta_d_C, f_C, is_f_value_default, region):
    if is_f_value_default == True :
        default_f_C = get_f_C_default(region)
        return Orientation(eta_d_C * default_f_C.D0, eta_d_C * default_f_C.D90, eta_d_C * default_f_C.D180, eta_d_C * default_f_C.D270)
    else:
        return Orientation(eta_d_C * f_C, eta_d_C * f_C, eta_d_C * f_C, eta_d_C * f_C)

#### Example

Use default value for f_C

In [34]:
vars(get_simple_eta_C_window(0.6, 1.0, True, 'region6'))

{'D0': 0.5112, 'D180': 0.5172, 'D270': 0.5112, 'D90': 0.5184}

Not use default value for f_C

In [35]:
vars(get_simple_eta_C_window(0.6, 1.0, False, 'region6'))

{'D0': 0.6, 'D180': 0.6, 'D270': 0.6, 'D90': 0.6}

#### Comment

①窓の暖房期の日射熱取得率($\eta_{H,wnd,SW}$・$\eta_{H,wnd,NW}$・$\eta_{H,wnd,NE}$・$\eta_{H,wnd,SE}$)  
窓の暖房期の日射熱取得率$\eta_{H,wnd,SW}$・$\eta_{H,wnd,NW}$・$\eta_{H,wnd,NE}$・$\eta_{H,wnd,SE}$は、当該住戸の窓の垂直面日射熱取得率を用いて第3章第4節「外皮の日射熱取得」に定める方法により計算した値とする。各部位で複数の異なる仕様を有する場合は、垂直面日射熱取得率が最も小さい部位の垂直面日射熱取得率の値とする。その際、暖房期の取得日射量補正係数$f_H$は、当該住戸で最も小さい値を用いるか、又は、ガラス区分を7、$l_1=0$、$l_2=1⁄0.3$として、地域の区分に応じて、南西、北西、北東及び南東の値を第3章第4節「外皮の日射熱取得」に定める方法により計算した値とする。ただし、窓の面積（当該窓が二以上の場合においては、その合計の面積。）が単位住戸の床面積に0.04を乗じた数値以下となるものを除くことができる。  

$\begin{align}l_1 = \frac{y_1}{z}\end{align}$  

$\begin{align}l_2 = \frac{(y_1+y_2)}{z}\end{align}$  

ここで、$l_1=0$から、$y_1=0.0$であり、$y_2=1.0$とすると$z=0.3$となる。  

$\begin{align}f &= \frac{ ( f_2 \times ( y_1 + y_2 ) － f_1 \times y_1 ) }{ y_2 } \\
                &= f_2 \end{align}$  

となる。

### 3.13 Direction Coefficient / 方位係数

#### Example

In [36]:
DC_get_nu_H('region1')['E']

0.564

In [37]:
DC_get_nu_C('region1')['E']

0.545

## 4. Integration

### 4.1 Input Data

````
spec = {
    'house_type' : 'base_ins', # 'floor_ins', 'base_ins' or 'floor_and_base_ins'
    'region'     : 'region6',  # 'region1', 'region2', ..., 'region8'
    'U' : {
        'roof'         : 0.240, # U value of roof (W/m2K)
        'wall'         : 0.530, # U value of wall (W/m2K)
        'door'         : 2.330, # U value of door (W/m2K)
        'window'       : 3.490, # U value of window (W/m2K)
        'floorOther'   : 0.480, # U value of floor of other part (W/m2K)
        'baseEntrance' : 0.000, # U value of base of entrance (W/m2K)
        'baseOther'    : 0.000, # U value of base of other part (W/m2K)
    },
    'psi' : {
        'useDefault'        : 'yes', # 'yes' or 'no', which are defalut 'perimeterEntrance' and 'perimeterOther' used or not.
        'perimeterEntrance' : 1.8,   # psi value of perimeter of entrance (W/mK),   which is not used in case that 'useDefault' is 'yes'.
        'perimeterOther'    : 1.8,   # psi value of perimeter of other part (W/mK), which is not used in case that 'useDefault' is 'yes'.
    },
    'eta_d' : {
        'heating' : 0.510, # eta d value in heating season
        'cooling' : 0.510  # eta d value in cooling season
    },
    'fValue' : {
        'useDefault' : 'yes', # 'yes' or 'no'  which is default f-value used or not 
        'heating'    : 1.0,   # f-value in heating season, which is not used in case that 'useDefault' is 'yes'
        'cooling'    : 1.0    # f-value in cooling season, which is not used in case that 'useDefault' is 'yes'
    }
}
````

### 4.2 Function

In [38]:
def simpleEnvPerformanceSelected(spec):

    def get_U_A_provisional(house_type, U, psi):
        A, L = getSimpleAreaAndLength(house_type)
        H = getSimpleHValue()
        return get_simple_U_A ( A = A, L = L, H = H, U = U, psi = psi )
    
    is_f_value_default = { 'yes' : True, 'no' : False }[spec['fValue']['useDefault']]
    
    spec['psi'] = setPsiPerimeter(spec['psi'])
    
    # U_A in case of both of floor insulated house and base insulated house
    U_A_floor_ins = get_U_A_provisional('floor_ins', spec['U'], spec['psi'])
    U_A_base_ins  = get_U_A_provisional('base_ins' , spec['U'], spec['psi'])

    # decide whether the floor insulated house or the base insulated house is applied for the evaluation,
    # when the actual house is the floor and base insulated house
    U_A, house_type_on_calc = {
        'floor_ins' : ( U_A_floor_ins, 'floor_ins' ),
        'base_ins'  : ( U_A_base_ins,  'base_ins'  ),
        'floor_and_base_ins' : judge_simple_house_type(U_A_floor_ins, U_A_base_ins)
    }[spec['house_type']]
    
    # area and length
    A, L = getSimpleAreaAndLength(house_type_on_calc)

    # eta in heating and cooling season
    etaHRoof,  etaCRoof  = get_simple_eta_not_window(spec['U']['roof'])
    etaHWall,  etaCWall  = get_simple_eta_not_window(spec['U']['wall'])
    etaHDoor,  etaCDoor  = get_simple_eta_not_window(spec['U']['door'])
    etaHBaseO, etaCBaseO = get_simple_eta_not_window(spec['U']['baseOther'])
    etaHBaseE, etaCBaseE = get_simple_eta_not_window(spec['U']['baseEntrance'])

    # r_env
    r_env = get_r_env(A['total_env'], A['total_floor'])

    # Q'
    Q_dash = get_Q_dash( float(U_A), r_env )

    # --- heating season ---
    if spec['region'] == 'region8':
        eta_A_H = 'ND'
        mu_H    = 'ND'
    else:
        # eta for windows
        etaHWindow = get_simple_eta_H_window(spec['eta_d']['heating'], spec['fValue']['heating'], is_f_value_default, spec['region'])
        # nu
        nuH = DC_get_nu_H(spec['region'])
        nuTopH, nuHorizontalH = ( nuH['top'], get_simple_Orientation_value_from_Direction(nuH) )
        # eta_A
        eta_A_H = get_simple_eta_A_H(A, nuTopH, nuHorizontalH, etaHRoof, etaHWall, etaHDoor, etaHWindow, etaHBaseO, etaHBaseE )
        # mu
        mu_H = get_mu_H(float(eta_A_H), r_env)
    
    # --- cooling season ---
    # eta for windows
    etaCWindow = get_simple_eta_C_window(spec['eta_d']['cooling'], spec['fValue']['cooling'], is_f_value_default, spec['region'])
    # nu
    nuC = DC_get_nu_C(spec['region'])
    nuTopC, nuHorizontalC = ( nuC['top'], get_simple_Orientation_value_from_Direction(nuC) )
    # eta_A
    eta_A_C = get_simple_eta_A_C(A, nuTopC, nuHorizontalC, etaCRoof, etaCWall, etaCDoor, etaCWindow, etaCBaseO, etaCBaseE )
    # mu
    mu_C = get_mu_C(float(eta_A_C), r_env)

    return {
        'UA' : U_A,
        'etaAH' : eta_A_H,
        'etaAC' : eta_A_C,
        'Qdash' : Q_dash,
        'muH'   : mu_H,
        'muC'   : mu_C
    }

#### Example

In [39]:
spec = {
    'house_type' : 'base_ins', # 'floor_ins', 'base_ins' or 'floor_and_base_ins'
    'region'     : 'region6',  # 'region1', 'region2', ..., 'region8'
    'U' : {
        'roof'         : 0.240, # U value of roof (W/m2K)
        'wall'         : 0.530, # U value of wall (W/m2K)
        'door'         : 2.330, # U value of door (W/m2K)
        'window'       : 3.490, # U value of window (W/m2K)
        'floorOther'   : 0.480, # U value of floor of other part (W/m2K)
        'baseEntrance' : 0.000, # U value of base of entrance (W/m2K)
        'baseOther'    : 0.000  # U value of base of other part (W/m2K)
    },
    'psi' : {
        'useDefault'        : 'yes',
        'perimeterEntrance' : 1.8, # psi value of perimeter of entrance (W/mK)
        'perimeterOther'    : 1.8  # psi value of perimeter of other part (W/mK)
    },
    'eta_d' : {
        'heating' : 0.510, # eta d value in heating season
        'cooling' : 0.510  # eta d value in cooling season
    },
    'fValue' : {
        'useDefault' : 'yes', # 'yes' or 'no'  which is default f-value used or not 
        'heating'    : 1.0,   # f-value in heating season, which is not used in case that 'UseDefault' is 'yes'
        'cooling'    : 1.0    # f-value in cooling season, which is not used in case that 'UseDefault' is 'yes'
    }
}

In [40]:
simpleEnvPerformanceSelected( spec = spec)    

{'Qdash': 2.8794288888888886,
 'UA': 0.94,
 'etaAC': 3.1,
 'etaAH': 3.3,
 'muC': 0.09495988888888889,
 'muH': 0.10108633333333332}

In [41]:
spec = {
    'house_type' : 'floor_ins', # 'floor_ins', 'base_ins' or 'floor_and_base_ins'
    'region'     : 'region1',  # 'region1', 'region2', ..., 'region8'
    'U' : {
        'roof'         : 7.700, # U value of roof (W/m2K)
        'wall'         : 6.670, # U value of wall (W/m2K)
        'door'         : 4.650, # U value of door (W/m2K)
        'window'       : 6.510, # U value of window (W/m2K)
        'floorOther'   : 5.270, # U value of floor of other part (W/m2K)
        'baseEntrance' : 0.000, # U value of base of entrance (W/m2K)
        'baseOther'    : 0.000  # U value of base of other part (W/m2K)
    },
    'psi' : {
        'useDefault'        : 'yes',
        'perimeterEntrance' : 1.8, # psi value of perimeter of entrance (W/mK)
        'perimeterOther'    : 1.8  # psi value of perimeter of other part (W/mK)
    },
    'eta_d' : {
        'heating' : 0.700, # eta d value in heating season
        'cooling' : 0.700  # eta d value in cooling season
    },
    'fValue' : {
        'useDefault' : 'yes', # 'yes' or 'no'  which is default f-value used or not 
        'heating'    : 1.0,   # f-value in heating season, which is not used in case that 'UseDefault' is 'yes'
        'cooling'    : 1.0    # f-value in cooling season, which is not used in case that 'UseDefault' is 'yes'
    }
}

In [42]:
simpleEnvPerformanceSelected( spec = spec)    

{'Qdash': 18.124366666666667,
 'UA': 6.13,
 'etaAC': 13.9,
 'etaAH': 14.7,
 'muC': 0.4109766666666667,
 'muH': 0.43463}

In [43]:
spec = {
    'house_type' : 'floor_ins', # 'floor_ins', 'base_ins' or 'floor_and_base_ins'
    'region'     : 'region8',  # 'region1', 'region2', ..., 'region8'
    'U' : {
        'roof'         : 7.700, # U value of roof (W/m2K)
        'wall'         : 6.670, # U value of wall (W/m2K)
        'door'         : 4.650, # U value of door (W/m2K)
        'window'       : 6.510, # U value of window (W/m2K)
        'floorOther'   : 5.270, # U value of floor of other part (W/m2K)
        'baseEntrance' : 0.000, # U value of base of entrance (W/m2K)
        'baseOther'    : 0.000  # U value of base of other part (W/m2K)
    },
    'psi' : {
        'useDefault'        : 'yes',
        'perimeterEntrance' : 1.8, # psi value of perimeter of entrance (W/mK)
        'perimeterOther'    : 1.8  # psi value of perimeter of other part (W/mK)
    },
    'eta_d' : {
        'heating' : 0.700, # eta d value in heating season
        'cooling' : 0.700  # eta d value in cooling season
    },
    'fValue' : {
        'useDefault' : 'yes', # 'yes' or 'no'  which is default f-value used or not 
        'heating'    : 1.0,   # f-value in heating season, which is not used in case that 'UseDefault' is 'yes'
        'cooling'    : 1.0    # f-value in cooling season, which is not used in case that 'UseDefault' is 'yes'
    }
}

In [44]:
simpleEnvPerformanceSelected( spec = spec)    

{'Qdash': 18.124366666666667,
 'UA': 6.13,
 'etaAC': 13.7,
 'etaAH': 'ND',
 'muC': 0.4050633333333334,
 'muH': 'ND'}